In [0]:
%sql
create catalog if not exists telecom_catalog_assign;

In [0]:
%sql
create schema if not exists telecom_catalog_assign.landing_zone

In [0]:
%sql
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol

In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

In [0]:
customer_csv = '''101,Arun,31,Chennai,PREPAID 
102,Meera,45,Bangalore,POSTPAID 
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID 
105,,27,Delhi,PREPAID 
106,Sneha,abc,Pune,PREPAID'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5 
103\t540\t600\t52 
104\t45\t200\t2
105\t0\t0\t0'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12'''

In [0]:
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.csv",customer_csv,True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.csv",usage_tsv,True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv",tower_logs_region1,True)

In [0]:
%fs ls /Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/



In [0]:
%fs ls /Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/


In [0]:
%fs ls /Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/

In [0]:
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.getOrCreate()
df=spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/"],pathGlobFilter="tower_logs*",recursiveFileLookup=True,sep='|',inferSchema=True,header=True)
df.show()

In [0]:
df=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.csv",header=True,inferSchema=True)
df.show()
#df.printSchema()


In [0]:
df=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.csv",header=False,inferSchema=True)
df.show()

In [0]:
df=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.csv",inferSchema=True,header=False,sep=',').toDF("id","name","age","loc","payment_type")
df.show()

In [0]:
df=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.csv",sep='\t',header=True)
df.show()

In [0]:
sch="id string,voice integer,data integer,sms_count string"
df=spark.read.schema(sch).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.csv",sep="\t",header=True)
df.show()

# df=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.csv",sep="\t",header=True,inferSchema=True)
# df.printSchema()


In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
sch=StructType([StructField("id",StringType(),True),StructField("voice",IntegerType(),True),StructField("data",IntegerType(),True),StructField("sms_count",StringType(),True)])
df=spark.read.schema(sch).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.csv",sep="\t",header=True)
# df.show()
df.printSchema()